In [1]:
import torch
from pykeops.torch import LazyTensor

from experiments.data_processing.load_torch import LOADERS
from experiments.utils import get_gp_hparams

In [2]:
seed = 0
torch.manual_seed(seed)

In [3]:
split_proportion = 0.1
split_shuffle = True
standardize = True
dtype = torch.float32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
load_fn = LOADERS["yolanda"]

In [5]:
dataset = load_fn(
    split_proportion=split_proportion,
    split_shuffle=split_shuffle,
    split_seed=seed,
    standardize=standardize,
    dtype=dtype,
    device=device,
)

In [6]:
gp_hparams = get_gp_hparams("yolanda", "rbf", seed).to(
    device=device, dtype=dtype)

In [7]:
# Create sample data
# n, d = 300000, 100
# x = torch.randn(n, d)
# y = torch.randn(n, d)
x = dataset.Xtr
y = dataset.Xtr
n, d = x.shape

# Convert to LazyTensors
x_i = LazyTensor(x[:, None, :])  # (n, 1, d)
y_j = LazyTensor(y[None, :, :])  # (1, n, d)

# Create scaling factors
# scales = torch.tensor([0.5, 2.0, 1.5])  # Different scale for each dimension
# scales = torch.randn(d, device=device) ** 2  # Random scales for each dimension
scales = gp_hparams.kernel_lengthscale

# Reshape scales to work with LazyTensors
scales_k = LazyTensor(scales[None, None, :])  # (1, 1, d)

# Scale columns
scaled_x_i = x_i * scales_k
scaled_y_j = y_j * scales_k

# Use in a computation (e.g., squared Euclidean distance with scaled columns)
D_ij = ((scaled_x_i - scaled_y_j) ** 2).sum(dim=2)
D_ij  = -D_ij / 2
D_ij = D_ij.exp()

In [8]:
w = torch.zeros(n, device=device, dtype=torch.float32)
print(D_ij @ w)

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
